In [1]:
from filelock import FileLock

In [5]:
from pathlib import Path
from typing import Dict

for folder_path in Path('output').glob('*'):
    if not folder_path.is_dir() or folder_path.name == '.ipynb_checkpoints':
        continue
    seq_path = folder_path / 'seqs'
    assert seq_path.exists()
    for path in seq_path.glob('*.fa'):
        ...

In [6]:
for path in Path('output/hairpin/seqs').glob('*.fa'):
    

PosixPath('output/linker_huge/seqs/linker_huge_5.fa')

In [66]:
from pathlib import Path
from typing import Dict


def parse_fasta(path: Path) -> Dict[str, str]:
    """
    read fasta seqs
    """
    fastadex = {}
    stem = path.stem
    with path.open() as file:
      indices = iter('ØABCDEF')
      index = next(indices)
      header = None
      sequence = []
    
      for line in file:
          if line.startswith('>'):
              if header and sequence:
                  fastadex[f"{stem}_{index}"] = ''.join(sequence).replace(" ", "")
                  index = next(indices)
                  sequence = []
              header = line.strip()
          else:
              sequence.append(line.strip())
    
      # Add the last sequence if the file ends without a new header
      if header and sequence:
          fastadex[f"{stem}{index}"] = ''.join(sequence).replace(" ", "")
    return fastadex

fastadex = {}
for path in Path('output/hairpin/seqs').glob('*.fa'):
    fastadex.update( parse_fasta(path) )

print(len(fastadex))

6000


In [67]:
import pandas as pd

fore = 'SRYFIEFEELQLLGKGAFGAVIKVQNKLDGCCYAVKRIPINPASRQFRRIKGEVTLLSRLHHENIVRYYNAWIE'
aft  = 'YLYIQMEYCEASTLRDTIDQGLYRDTVRLWRLFREILDGLAYIHEKGMIHRNLKPVNIFLDSDDHVKIGQKVDLFSLGIIFFEMSYHPMVTASERIFVLNQLRDPTSPKFPEDFDDGEHAKQKSVISWLLNHDPAKRPTATELLKSELLPPP'

df = pd.DataFrame({'Name': fastadex.keys(), 'seq': fastadex.values()}).sort_values('Name').copy()
df['ins'] = df.seq.str.replace(fore, '').str.replace(aft, '')

In [23]:
import sys

#sys.path.append('/opt/xchem-fragalysis-2/mferla/crysalin-redux/repo/code')
import thread_tune_v2 as tt

In [34]:
df.ins.value_counts().head(10)

ins
DG       1254
EEEG      380
PSG       344
GG        279
EETG      244
GGGGG     243
GGGG      241
DDG       240
GGG       238
EESG      166
Name: count, dtype: int64

In [39]:
top_ins = 'DG'
row = df.loc[df.ins == top_ins].iloc[0]
target_name = row.Name
target_sequence = row.seq

In [ ]:
SETTINGS['start_seqs'] = ['SRYFIEF']
SETTINGS['chain_letters'] = 'A'

result = run_process(target_folder=Path('output/hairpin'),
                          target_name=target_name,
                          target_sequence=target_sequence,
                          hallucination_name=target_name[:-2],
                          parent_filename=Path('6N3O_relaxed_trimmed.pdb'),
                          metadata={},
                          override=True,
                          )
print(result)
print('\n'.join(result['traceback']))

In [103]:
!cp output/hairpin/hairpin_0.pdb copies/hairpin_0.pdb

In [102]:
!echo 'THESE_ARE_TEMP' >> copies/THESE_ARE_TEMP

In [123]:
!rm output/hairpin/log.jsonl

In [73]:
import pickle

with open('output/hairpin/hairpin_0.trb', 'rb') as fh:
    trb = pickle.load(fh)
trb['complex_con_ref_idx0'] = trb['con_ref_idx0']
trb['complex_con_hal_idx0'] = trb['con_hal_idx0']
with open('output/hairpin/hairpin_0.trb', 'wb') as fh:
    pickle.dump(trb, fh)

In [106]:
pose = pyrosetta.pose_from_file('output/hairpin/hairpin_0.pdb')

In [122]:
def appraise_itxns(pose,
                   max_clashes=0,
                   clash_dist_cutoff=1.5,
                   bond_dist_cutoff=1.7,
                   trb: Optional[Dict[str, Any]] = None
                   ) -> Tuple[int, int]:
    """
    Assumes the chains have been fixed already.
    if `trb` is not None, the reference is used to check only stretching in the designed bits.
    NB. if the PDB had a gap and no trb is passed, it will be considered a stretch!

    :param pose:
    :param max_clashes:
    :param clash_dist_cutoff:
    :param bond_dist_cutoff:
    :param trb:
    :return:
    """
    n_clashing = 0
    chains = pose.split_by_chain()
    xyz_model = extract_coords(chains[1])
    for idx0 in range(1, pose.num_chains()):  # chain 0 is the full AHIR, designed
        xyz_other = extract_coords(chains[idx0 + 1])
        distances = np.sqrt(np.sum((xyz_other[:, np.newaxis, :] - xyz_model[np.newaxis, :, :]) ** 2, axis=-1))
        # 1.5 Å is too close
        n_clashing += np.count_nonzero(distances < clash_dist_cutoff)
    if n_clashing > max_clashes:
        raise ValueError(f'{n_clashing} clashes')
    # check no stretch
    n_warning_stretch = 0
    if trb is None:
        termini = [pose.chain_begin(c+1) for c in range(pose.num_chains())] + \
                  [pose.chain_end(c+1) for c in range(pose.num_chains())]
        # I could be doing pose.residue(i).is_terminus() but that is a needless amount of calls
        indices1 = [idx0+1 for idx0 in range(pose.total_residue() - 1) if idx0+1 not in termini]
    else:
        # get the not conserved and the preceding residue (unless it is the first)
        indices1 = [idx0 + 1 for idx0, conned in enumerate(trb['inpaint_seq']) if not conned] + \
                   [idx0 for idx0, conned in enumerate(trb['inpaint_seq']) if not conned if idx0 != 0]
        indices1 = list(set(indices1))
    for idx1 in indices1:
        d: float = pose.residue(idx1).xyz('C').distance(pose.residue(idx1 + 1).xyz('N'))
        if d > bond_dist_cutoff:
            raise ValueError(f'Stretch {d:.1}')
        if d > 1.36:
            n_warning_stretch += 1

tt.appraise_itxns = appraise_itxns

In [ ]:
target_folder = Path('output/hairpin/seqs')
parent_filename:Path('')
timeout = 60 * 60
seq_paths = [se]
    futures: List[ProcessFuture] = []
    future_names: List[str] = []
    with ProcessPool(max_workers=get_max_cores() - 1, max_tasks=0) as pool:
        # queue jobs
        for path in seq_paths:
            # send each seq to the pool
            for seq_record in SeqIO.parse(path, 'fasta'):
                # ## Read metadata
                metadata = {k: float(v) for k, v in re.findall(r'([\w_]+)=([\d.]+)', seq_record.description)}
                hallucination_name: str = path.stem  # noqa
                target_sequence: str = str(seq_record.seq) # noqa
                target_name = f"{hallucination_name}{'ABCDEFGHIJKLMNOPQRSTUVWXYZØ'[int(metadata.get('sample', -1))]}"
                future: ProcessFuture = pool.schedule(run_process,
                                                      # target = seq-variant of hallucination
                                                      # parent = WT template
                                                      # hallucination = RFdiffused skeleton
                                                      kwargs=dict(target_folder=target_folder,
                                                                  target_name=target_name,
                                                                  target_sequence=target_sequence,
                                                                  hallucination_name=hallucination_name,
                                                                  parent_filename=parent_filename,
                                                                  metadata=metadata,
                                                                  ),
                                                      timeout=timeout)

                future_names.append(target_name)
                futures.append(future)
        print(f'Submitted {len(futures)} processes')
        # ## Get results
        results: List[dict] = []
        for name, future in zip(future_names, futures):
            try:
                result = future.result()  # blocks until results are ready
                print(result['name'], result['status'])
                results.append(result)
            except Exception as error:
                error_msg = str(error)
                result = dict(target_name=name, error=str(error), error_type=error.__class__.__name__,)
                results.append(result)
                if isinstance(error, TimeoutError):
                    print(f'Function took longer than {timeout} seconds {error}')
                else:
                    print(f"Function raised {error}")
                    traceback.print_tb(error.__traceback__)  # traceback of the function
    df = pd.DataFrame(results)
    df.to_pickle(target_folder / 'tuned.pkl.gz')
    return df

In [96]:
common_pyrosetta.steal_frozen = steal_frozen

In [ ]:
tt.steal_frozen = stea

In [153]:
cycles = 1
scorefxn_cart = pyrosetta.create_score_function('ref2015_cart')
assert scorefxn_cart.get_weight(pr_scoring.ScoreType.cart_bonded) > 0., 'Cartesian required'
scorefxn = pyrosetta.create_score_function('ref2015')
print(scorefxn(pose), scorefxn_cart(pose) )
relax = prp.relax.FastRelax(scorefxn_cart, cycles)
relax.cartesian(True)
relax.minimize_bond_angles(True)
relax.minimize_bond_lengths(True)
relax.apply(pose)
print(scorefxn(pose), scorefxn_cart(pose) )

27408.372480047885 88012.67651372956
29644.28177443537 90385.22366141704


In [ ]:
cart_bonded to 0.5 and pro_close 0.0



In [ ]:
pose = pyrosetta.pose_from_file('output/hairpin/hairpin_0.pdb')

    for chain in chains:
        for i in range(chain.total_residue() - 1):
            d: float = chain.residue(i + 1).xyz('C').distance(chain.residue(i + 1 + 1).xyz('N'))
            if d > bond_dist_cutoff:
                raise ValueError(f'Stretch {d:.1}')
            if d > 1.36:
                n_warning_stretch += 1

In [ ]:
    for chain in chains:
        for i in range(chain.total_residue() - 1):
            d: float = chain.residue(i + 1).xyz('C').distance(chain.residue(i + 1 + 1).xyz('N'))
            if d > bond_dist_cutoff:
                raise ValueError(f'Stretch {d:.1}')
            if d > 1.36:
                n_warning_stretch += 1